In [1]:
import torch
import numpy as np

from datasets.dataset import transform_dataset, kfold_dataset
from DRNet import train, DRNet

import sys

/vol/home/s2299925/miniconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/vol/home/s2299925/miniconda3/envs/thesis/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/vol/home/s2299925/miniconda3/envs/thesis/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c104impl8GPUTrace13gpuTraceStateE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Read datasets
name = 'adult'
X, Y, X_headers, Y_headers = transform_dataset(name, method='onehot-compare', negations=False, labels='binary')
datasets = kfold_dataset(X, Y, shuffle=1)
X_train, X_test, Y_train, Y_test = datasets[0]
train_set = torch.utils.data.TensorDataset(torch.Tensor(X_train.to_numpy()), torch.Tensor(Y_train))
test_set = torch.utils.data.TensorDataset(torch.Tensor(X_test.to_numpy()), torch.Tensor(Y_test))

In [3]:
# Train DR-Net
# Default learning rate (1e-2), and_lam (1e-2), and and_lam (1e-5) usually work the best. A large epochs number is necessary for a sparse rule set i.e 10000 epochs.
net = DRNet(train_set[:][0].size(1), 50, 1)
train(net, train_set, test_set=test_set, device='cpu', lr=1e-2, epochs=1000, batch_size=400,
      and_lam=1e-2, or_lam=1e-5, num_alter=500)

Epoch: 100%|██████████| 1000/1000 [17:30<00:00,  1.05s/it, loss=0.549, epoch acc


In [4]:
# Get accuracy and the rule net
accu = (net.predict(np.array(X_test)) == Y_test).mean()
rules = net.get_rules(X_headers)
print(f'Accuracy: {accu}, num rules: {len(rules)}, num conditions: {sum(map(len, rules))}')

Accuracy: 0.8266202552627216, num rules: 50, num conditions: 1800


In [5]:
print(rules)

[['NOT age<=22.0', 'NOT age<=26.0', 'NOT age<=30.0', 'NOT age<=33.0', 'NOT education==10th', 'NOT education==11th', 'NOT education==12th', 'NOT education==5th-6th', 'NOT education==7th-8th', 'NOT education==9th', 'NOT education==Assoc-acdm', 'NOT education==Assoc-voc', 'NOT education==Preschool', 'NOT education_num<=7.0', 'education_num<=9.0', 'NOT education_num<=11.0', 'NOT marital-status==Divorced', 'marital-status==Married-civ-spouse', 'NOT marital-status==Married-spouse-absent', 'NOT marital-status==Never-married', 'NOT marital-status==Separated', 'NOT marital-status==Widowed', 'NOT occupation==Craft-repair', 'NOT occupation==Farming-fishing', 'NOT occupation==Handlers-cleaners', 'NOT occupation==Machine-op-inspct', 'NOT occupation==Other-service', 'NOT occupation==Priv-house-serv', 'NOT occupation==Transport-moving', 'NOT relationship==Not-in-family', 'NOT relationship==Other-relative', 'NOT relationship==Own-child', 'NOT relationship==Unmarried', 'sex', 'NOT capital-gain<=0.0', '